# Test Cases

This notebook contains all the testcases that our API supports.

### Required Installations

In [1]:
!pip install -U ipytest
!pip install pytest-tornasync
!pip install nest_asyncio
!pip install ipynb

### Importing the required modules

In [2]:
import nest_asyncio
nest_asyncio.apply()

import ipytest
ipytest.autoconfig()

import pytest
from fastapi.testclient import TestClient
from fastapi import Depends
from fastapi.security.api_key import APIKey
import warnings
from ipynb.fs.full.main import *

### Initializing a TestClient

In [3]:
client = TestClient(app)

## Test Case1: 
**Function:** 
1. Prints Welcome message in the Root page 

**Test:** 
1. Checking the response code if it is 200; The HTTP Status 200 (OK) status code indicates that the request has been processed successfully on server
2. Checking if the output is same as the Welcome message

In [4]:
%%run_pytest[clean]

def test_read_root():
    warnings.simplefilter("always")
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"Welcome to Team 6 Project Page": "Production Plant Data for Condition Monitoring"}

.                                                                        [100%]
1 passed in 0.09s


## Test Case2: 
**Function:** 
1. Getting the complete data from Snowflake i.e. all 27 columns.

**Test:** 
1. Checking the response code if it is 200; The HTTP Status 200 (OK) status code indicates that the request has been processed successfully on server
2. Checking if the length of the response is as expected. For testing, we are querying data with limit 5 so here expected length would be 5.

In [5]:
%%run_pytest[clean]

def test_fetch_all_data(api_key: APIKey = Depends(get_api_key)):
    response = client.get("/ProductionPlantData?access_token=Team6")
    assert response.status_code == 200
    assert len(response.json()) == 5

.                                                                        [100%]
1 passed in 0.60s


## Test Case3: 
**Function:** 
1. Getting the data from Snowflake based on the column name(s) given as parameter

**Parameter Passed:**
1. Column which is present in the table A_1

**Test:** 
1. Checking the response code if it is 200; The HTTP Status 200 (OK) status code indicates that the request has been processed successfully on server
2. Checking the column name which is given as parameter is present in the table
3. Checking if the length of the response is as expected


In [6]:
%%run_pytest[clean]

def test_fetch_data_by_columns(api_key: APIKey = Depends(get_api_key)):
    column = "A_1"
    response = client.get("/ProductionPlantData/column/{}?access_token=Team6".format(column))
    assert response.status_code == 200
    assert (column in response.json().keys()) == True
    assert len(response.json()[column]) == 6

.                                                                        [100%]
1 passed in 0.24s


## Test Case4: 
**Function:** 
1. Getting the data from Snowflake based on the column name(s) given as parameter

**Parameter Passes:**
1. Column not present in table i.e M_1

**Test:** 
1. Checking the response code if it is 404
3. Checking if the output is "One or more columns not Found"

In [7]:
%%run_pytest[clean]

def test_fetch_data_by_columns(api_key: APIKey = Depends(get_api_key)):
    column = "M_1"
    response = client.get("/ProductionPlantData/column/{}?access_token=Team6".format(column))
    assert response.status_code == 404
    assert response.json() == {"detail":"One or more columns not Found"}

.                                                                        [100%]
1 passed in 0.24s


## Test Case5: 
**Function:** 
1. Getting the data from Snowflake based on the experiment given as parameter

**Parameter Passes:**
1. Experiment present in table i.e 7

**Test:** 
1. Checking the response code if it is 200; The HTTP Status 200 (OK) status code indicates that the request has been processed successfully on server
3. Checking if the length of the response is as expected

In [8]:
%%run_pytest[clean]

def test_get_data_by_experiment(api_key: APIKey = Depends(get_api_key)):
    experiment = 7
    response = client.get("/ProductionPlantData/experiment/{}?access_token=Team6".format(experiment))
    assert response.status_code == 200
    assert len(response.json()) == 2

.                                                                        [100%]
1 passed in 0.16s


## Test Case6: 
**Function:** 
1. Getting the data from Snowflake based on the experiment given as parameter

**Parameter Passes:**
1. Experiment which is not present in table i.e 81

**Test:** 
1. Checking the response code if it is 404
3. Checking the output if it is "Experiment Number is not found"

In [9]:
%%run_pytest[clean]

def test_get_data_by_experiment(api_key: APIKey = Depends(get_api_key)):
    experiment = 81
    response = client.get("/ProductionPlantData/experiment/{}?access_token=Team6".format(experiment))
    assert response.status_code == 404
    assert response.json() == {"detail":"Experiment Number not Found"}

.                                                                        [100%]
1 passed in 0.05s


## Test Case7: 
**Function:** 
1. Getting the data from Snowflake based on the Timestamp given as parameter

**Parameter Passes:**
1. Timestamp which is present in table i.e 1000

**Test:** 
1. Checking the response code if it is 200; The HTTP Status 200 (OK) status code indicates that the request has been processed successfully on server
3. Checking if the length of the response is as expected

In [10]:
%%run_pytest[clean]

def test_get_data_by_timestamp(api_key: APIKey = Depends(get_api_key)):
    timestamp = 1000
    response = client.get("/ProductionPlantData/timestamp/{}?access_token=Team6".format(timestamp))
    assert response.status_code == 200
    assert len(response.json()) == 10

.                                                                        [100%]
1 passed in 0.30s


## Test Case8: 
**Function:** 
1. Getting the data from Snowflake based on the experiment given as parameter

**Parameter Passes:**
1. Timestamp which is not present in table i.e 99999

**Test:** 
1. Checking the response code if it is 404
3. Checking the output if it is "Timestamp not found"

In [11]:
%%run_pytest[clean]

def test_get_data_by_timestamp(api_key: APIKey = Depends(get_api_key)):
    timestamp = 99999
    response = client.get("/ProductionPlantData/timestamp/{}?access_token=Team6".format(timestamp))
    assert response.status_code == 404
    assert response.json() == {"detail":"Timestamp not Found"}

.                                                                        [100%]
1 passed in 0.14s


## Test Case9: 
**Function:** 
1. Getting the data from Snowflake based on the features given as parameter 
(There are 26 columns which are combined into 8 features)

**Parameter Passes:**
1. Feature which is present in table i.e feature_1

**Test:** 
1. Checking the response code if it is 200
2. Checking if the length of the response is as expected

In [12]:
%%run_pytest[clean]

def test_get_data_by_feature(api_key: APIKey = Depends(get_api_key)):
    feature = 'feature_1'
    response = client.get("/ProductionPlantData/feature/{}?access_token=Team6".format(feature))
    assert response.status_code == 200
    assert len(response.json()) == 3

.                                                                        [100%]
1 passed in 0.23s


## Test Case10: 
**Function:** 
1. Getting the data from Snowflake based on the features given as parameter

**Parameter Passes:**
1. Feature which is not present in table i.e feature_10

**Test:** 
1. Checking the response code if it is 404
3. Checking the output if it is "Feature not Found"

In [13]:
%%run_pytest[clean]

def test_get_data_by_feature(api_key: APIKey = Depends(get_api_key)):
    feature = 'feature_10'
    response = client.get("/ProductionPlantData/feature/{}?access_token=Team6".format(feature))
    assert response.status_code == 404
    assert response.json() == {"detail":"Feature not Found"}

.                                                                        [100%]
1 passed in 0.04s


## Test Case11: 
**Function:** 
1. Getting the data from Snowflake based on the column and the range given as parameter

**Parameter Passes:**
1. Column Name which is present in table i.e A_1 
2. Range = 23 - 33

**Test:** 
1. Checking the response code if it is 200
2. Checking if the length of the response is as expected

In [14]:
%%run_pytest[clean]

def test_get_data_between_range(api_key: APIKey = Depends(get_api_key)):
    column_name = 'A_1'
    data_greater_than = 23
    data_less_than = 33
    response = client.get("/ProductionPlantData/data_between/{0}/{1}/{2}?access_token=Team6".format(column_name, data_greater_than, data_less_than))
    assert response.status_code == 200
    assert len(response.json()) == 5

.                                                                        [100%]
1 passed in 0.18s


## Test Case 12: 
**Function:** 
1. Getting the data from Snowflake based on the column and the range given as parameter

**Parameter Passes:**
1. Column Name which is not present in table i.e M_1
2. Range = 23 - 33

**Test:** 
1. Checking the response code if it is 404
2. Checking the output if it is "Variables not Found"

In [15]:
%%run_pytest[clean]

def test_get_data_between_range(api_key: APIKey = Depends(get_api_key)):
    column_name = 'Z_11'
    data_greater_than = 23
    data_less_than = 33
    response = client.get("/ProductionPlantData/data_between/{0}/{1}/{2}?access_token=Team6".format(column_name, data_greater_than, data_less_than))
    assert response.status_code == 404
    assert response.json() == {"detail":"Variables not Found"}

.                                                                        [100%]
1 passed in 0.23s
